# Deep Percolation Event Analysis and Visualization

This Jupyter Notebook is designed to analyze and visualize deep percolation events in lysimeter data, incorporating meteorological and soil measurements. The analysis focuses on understanding the characteristics of these events, their seasonal variations, and the factors influencing percolation under different conditions.

## Key Analyses and Visualizations

1. **Warm vs. Cold Season Event Comparison:**
   * Identifies deep percolation events as "warm" or "cold" based on snow cover and temperature conditions.
   * Compares event characteristics (volume, duration, intensity) between warm and cold seasons using boxplots.

2. **Correlation Analysis:**
   * Calculates Pearson and partial correlation coefficients between percolation event volume and various factors (water input, intensity index, soil moisture, soil temperature) for both warm and cold seasons.
   * Presents correlation results in a visually clear plot, highlighting significant relationships.

3. **Event Volume and Duration Distributions:**
   * Generates histograms and kernel density estimates (KDEs) to visualize the distributions of percolation event volumes and durations.
   * Provides summary statistics (mean, median, standard deviation) for these distributions.

4. **General and Seasonal Event Statistics:**
   * Reports key statistics (min, max, mean, median, coefficient of variation) for event volumes and durations.
   * Calculates seasonal averages for percolation volume, rainfall volume, and snowmelt contribution.
   * Presents seasonal variations in a bar plot, illustrating the relative importance of different water sources.

5. **Detailed Event Visualization:**
   * Selects specific events of interest (e.g., Event-9, Event-47, Event-34).
   * Creates detailed time series plots for each selected event, showing:
      * Hourly rainfall, simulated water input, and snow depth.
      * Soil moisture and temperature at various depths.
      * Hourly percolation rates.
      * Air temperature.

## Data Sources

* **Lysimeter Data:** Hourly deep percolation measurements.
* **Meteorological Data:** Hourly and daily rainfall, snow depth, air temperature, etc.
* **Soil Data:** Half-hourly soil moisture, temperature, and suction.
* **SVS Output:** Simulated water input at the surface from the Soil-Vegetation-Snow (SVS) model.

## Dependencies

* Python 3.x
* pandas
* NumPy
* SciPy
* pingouin (for statistical analysis)
* Matplotlib
* Seaborn
* Plotly

## Usage

1. **Data Preparation:** Ensure all data files are in the specified directories .
2. **Run the Notebook:** Execute the cells sequentially to perform the analysis and generate the visualizations.
3. **Interpret Results:** Examine the plots and statistics to gain insights into deep percolation events and their influencing factors.

## Additional Notes

* The notebook includes helper functions (`user_functions.py`) for data loading, calculations, and plotting.
* Some sections of the notebook focus on specific events and may require modification if you want to analyze different events.
* The visualizations are saved as PDF files.


# Imports

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import pickle as pkl
from matplotlib import pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

import user_functions as uf

# Variables

In [ ]:
# provide the path to the data and the lysimeter objects
# note that this repo does not contain the data, you need to download it from the dataverse
# and provide the path to the data in the DATA_DIR
DATA_DIR = Path(
    "../Data/"
)

## this is the path to the dir where the lysimeter objects are saved (outputs from the other notebook)
RESULT_DIR = Path(
    "../Data/Lysimeter_Objects/"
)


paths = {
    "data_percolation": DATA_DIR / Path("FieldHourlyDeepPercolation_3Enclosures_4Lysimeters_201806_202210.csv"),
    "data_meteo_hourly": DATA_DIR / Path("HourlyMeteo_201707_202212.parquet"),
    "data_meteo_daily": DATA_DIR / Path("DailyMeteo_201707_202212.csv"),
    "data_soil_moist": DATA_DIR / Path("FieldSoilMoisture_4Enclosures_4Lysimeters_201806_202210.parquet"),
    "data_soil_temp": DATA_DIR / Path("FieldSoilTemperature_4Enclosures_4Lysimeters_201806_202210.parquet"),
    "data_soil_suction": DATA_DIR / Path("FieldSoilSuction_4Enclosures_4Lysimeters_201806_202210.parquet"),

    # lysimeter objects
    "L1_events": RESULT_DIR / Path("L1_events.pkl"),
    "L2_events": RESULT_DIR / Path("L2_events.pkl"),
    "L3_events": RESULT_DIR / Path("L3_events.pkl"),
    "L4_events": RESULT_DIR / Path("L4_events.pkl"),

    # svs output for the input water flux
    "svs_waterinput": DATA_DIR / Path("InputWater_SVSoutput_ensAve.csv"),

    # output dir for saving figures
    "fig_dir": Path("../Figures/"),
}


# required for getting simulated input water flux from svs
pre_event_window_hours = 24 * 4

LOCAL_TZ = "America/Montreal"

# set plotting parameters
sns.set_style("whitegrid")
sns.set_context("paper")
sns.set_style("whitegrid", {'axes.grid' : False})
plt.rcParams['font.weight'] = 'bold'
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titleweight'] = 'bold'


## colors
WARM_GREY = '#A9A9A9'
SLATE_GREY = '#708090'
LIGHT_GREY = "#D3D3D3"
CHARCOAL = '#36454F'
TEAL = '#008080'
GOLD_BROWN = '#CC9966'
LIGHT_BLUE = '#6699CC'

WARM_GREY = '#707070'  # Make this darker if using Option 2
SLATE_GREY = '#8090A0'  # Make this cooler if using Option 2

DARK_PURPLE = '#663366'

# assert
for path in paths.values():
    assert path.exists(), f"{path} does not exist"

# Load Data

## Field data

In [ ]:
# read percolation measurements
data_hourly_percolation = pd.read_csv(
    paths["data_percolation"], index_col=0, parse_dates=True
)

# read daily meteo data
data_meteo_daily = pd.read_csv(
    paths["data_meteo_daily"], index_col=0, parse_dates=True
)

# read hourly meteo data
data_meteo_hourly = pd.read_parquet(
    paths["data_meteo_hourly"], engine="pyarrow"
)

# read half-hourly soil moisture / temperature data
data_soil_moist = pd.read_parquet(
    paths["data_soil_moist"], engine="pyarrow"
)
data_soil_temp = pd.read_parquet(
    paths["data_soil_temp"], engine="pyarrow"
)
data_soil_suction = pd.read_parquet(
    paths["data_soil_suction"], engine="pyarrow"
)

# set index for parquet files
data_soil_moist["Date Time (UTC)"] = pd.to_datetime(
    data_soil_moist["Date Time (UTC)"], utc=True
)
data_soil_moist.set_index("Date Time (UTC)", inplace=True)

data_soil_temp["Date Time (UTC)"] = pd.to_datetime(
    data_soil_temp["Date Time (UTC)"], utc=True
)
data_soil_temp.set_index("Date Time (UTC)", inplace=True)
data_soil_suction["Date Time (UTC)"] = pd.to_datetime(
    data_soil_suction["Date Time (UTC)"], utc=True
)
data_soil_suction.set_index("Date Time (UTC)", inplace=True)

data_meteo_hourly["Date Time (UTC)"] = pd.to_datetime(
    data_meteo_hourly["Date Time (UTC)"], utc=True
)
data_meteo_hourly.set_index("Date Time (UTC)", inplace=True)

# remove any negative suction values
data_soil_suction[data_soil_suction['Reading'] < 0] = np.nan

## correction
data_soil_suction.loc[
    (data_soil_suction["Unique_label_"] == "L1_1850mm_int_Suction_kPa") &
    (data_soil_suction.index.get_level_values(0) >= "2021-05-29 09:00:00") &
    (data_soil_suction.index.get_level_values(0) <= "2021-05-29 10:00:00"),
    "Reading"
] = np.nan

data_soil_suction.loc[
    (data_soil_suction["Unique_label_"] == "L4_1850mm_int_Suction_kPa") &
    (data_soil_suction.index.get_level_values(0) >= "2021-05-07 10:00:00") &
    (data_soil_suction.index.get_level_values(0) <= "2021-05-07 13:00:00"),
    "Reading"
] = np.nan

print("Data loaded successfully")

## read lysimeter objects

In [ ]:
# load all events objects
L1_events = uf.load_events(paths["L1_events"])
L2_events = uf.load_events(paths["L2_events"])
L3_events = uf.load_events(paths["L3_events"])
L4_events = uf.load_events(paths["L4_events"])

# from each event object, we need different columns from different dataframes


## these are the cols we need from the event dataframe
percolation_events_cols = {
    "percolation_start_dt": ("event_start_dt", "datetime64[ns, UTC]"),
    "percolation_end_dt": ("event_end_dt", "datetime64[ns, UTC]"),
    "percolation_volume_mm": ("event_volume_mm", "float64"),
    "percolation_duration_h": ("event_duration_h", "float64"),
    "percolation_max_intensity_mmh": ("event_max_rate_mmh", "float64"),
    "percolation_ave_intensity_mmh": ("event_avg_rate_mmh", "float64"),
}

## these are the cols we need from the rainfall dataframe
rainfall_events_cols = {
    "rainfall_volume_mm": ("rainfall_volume_mm", "float64"),
}

## these are the cols we need from the snow dataframe
snow_events_cols = {
    "ave_snowdepth_cm": ("ave_depth_cm", "float64"),
    "ave_snowdepth_prewindow_cm": ("ave_depth_prewindow_cm", "float64"),
    "snowcover_present_period_fraction": ("snowcover_present_period_fraction", "float64"),
}

## these are the cols we need from the surface dataframe
surface_soil_cols = {
    "surface75mm_antecedent_SoilMoisture_percent": ("prior_vwc", "float64"),
    "surface75mm_antecedent_SoilTemp_celsius": ("prior_temp", "float64"),
}


## the process for creating a dataframe that have all the required columns ...
column_types = {k: v[1] for k, v in percolation_events_cols.items()}
column_types.update({k: v[1] for k, v in rainfall_events_cols.items()})
column_types.update({k: v[1] for k, v in snow_events_cols.items()})
column_types.update({k: v[1] for k, v in surface_soil_cols.items()})

### Create the DataFrame with specified data types
percolation_events = pd.DataFrame({
    col: pd.Series(dtype=typ) for col, typ in column_types.items()
})

### loop over all events and the associated varaibles and add them to the dataframe
for data in [L1_events.events_df, L2_events.events_df, L3_events.events_df, L4_events.events_df]:
    for event_label, row in data.iterrows():
        for col in percolation_events_cols:
            percolation_events.loc[event_label, col] = row[percolation_events_cols[col][0]]


for data in [L1_events.rainfall_events_df, L2_events.rainfall_events_df, L3_events.rainfall_events_df, L4_events.rainfall_events_df]:
    for event_label, row in data.iterrows():
        for col in rainfall_events_cols:
            percolation_events.loc[event_label, col] = row[rainfall_events_cols[col][0]]

for data in [L1_events.snow_events_df, L2_events.snow_events_df, L3_events.snow_events_df, L4_events.snow_events_df]:
    for event_label, row in data.iterrows():
        for col in snow_events_cols:
            percolation_events.loc[event_label, col] = row[snow_events_cols[col][0]]

for data in [L1_events.surface_soil_df, L2_events.surface_soil_df, L3_events.surface_soil_df, L4_events.surface_soil_df]:
    for event_label, row in data.iterrows():
        for col in surface_soil_cols:
            value = row[surface_soil_cols[col][0]]
            if pd.isna(value):  # Check if it's NaT or another missing value
                value = np.nan     # Replace NaT with 0.0 (or another suitable float)
            percolation_events.loc[event_label, col] = value

percolation_events["event_label"] = percolation_events.index

### order by start date
percolation_events = percolation_events.sort_values(by="percolation_start_dt")

### reset index and rename
percolation_events.reset_index(drop=True, inplace=True)
percolation_events.index = [f"Event-{i}" for i in range(1, len(percolation_events) + 1)]
percolation_events.index.name = "event_id"

# for each event, we need a col `AirTemperature_prewindow_celsius`:
# this is the average air temperature in the 24*4 hours before the event
# we will use the hourly meteo data for this
percolation_events["AirTemperature_prewindow_celsius"] = np.nan
for event_id, event in percolation_events.iterrows():
    t0 = event["percolation_start_dt"] - pd.Timedelta(hours=pre_event_window_hours)
    t1 = event["percolation_start_dt"]
    pre_event_window_temp = data_meteo_hourly.loc[
        data_meteo_hourly["Variable"] == "Air temperature", "Reading"
    ].loc[t0:t1]

    percolation_events.loc[event_id, "AirTemperature_prewindow_celsius"] = pre_event_window_temp.mean()

# fill na with np.nan
num_cols = [
    "surface75mm_antecedent_SoilMoisture_percent",
    "surface75mm_antecedent_SoilTemp_celsius"
]
percolation_events[num_cols] = percolation_events[num_cols].fillna(np.nan)
percolation_events[num_cols] = percolation_events[num_cols].astype("float64")

print("Percolation events dataframe created successfully")

## Read the svs output for the input water flux

In [ ]:
# this is needed to get the SVS simulated water reaching surface
svs_ens_ave = pd.read_csv(paths["svs_waterinput"], index_col=0, parse_dates=True)
svs_ens_ave.index = pd.to_datetime(svs_ens_ave.index, utc=True)

# add it as a column for each event*
# * any time step that we do not have snow cover, this variable will be equal to the rainfall
percolation_events["SVS_water_input_mm"] = np.nan # initialize


# simple hourly intensity index for SVS_water_input_mm
percolation_events["SHII"] = np.nan # initialize


for event_label, row in percolation_events.iterrows():

    t0 = row["percolation_start_dt"] - pd.Timedelta(hours=pre_event_window_hours)
    t1 = row["percolation_end_dt"]
    percolation_events.loc[event_label, "SVS_water_input_mm"] = svs_ens_ave.loc[t0:t1, "Water input at surface (mm/h)"].sum()
    percolation_events.loc[event_label, "SVS_water_input_kurtosis"] = svs_ens_ave.loc[t0:t1, "Water input at surface (mm/h)"].kurtosis()
    percolation_events.loc[event_label, "SVS_water_input_skewness"] = svs_ens_ave.loc[t0:t1, "Water input at surface (mm/h)"].skew()

    percolation_events.loc[event_label, "SHII"] = (
        svs_ens_ave.loc[t0:t1, "Water input at surface (mm/h)"].sum() /
        (svs_ens_ave.loc[t0:t1, "Water input at surface (mm/h)"] > 0.0).sum()
    )


# for any event where snowcover_present_period_fraction == 0, set SVS_water_input_mm to rainfall_volume_mm
percolation_events.loc[
    percolation_events["snowcover_present_period_fraction"] == 0.0, "SVS_water_input_mm"
] = percolation_events.loc[
    percolation_events["snowcover_present_period_fraction"] == 0.0, "rainfall_volume_mm"
]

print("SVS water input added to the percolation events dataframe")

# General comparison of cold-season and warm-season events

In [ ]:
# the index of cold-season events based on the defined criteria:
# 1. snow depth > 1 cm OR 2. air temperature < 0
cold_events = percolation_events.loc[
    (percolation_events["ave_snowdepth_prewindow_cm"] > 1) |
    (percolation_events["AirTemperature_prewindow_celsius"] < 0)
].index

# add Event-51 based on individual inspection
cold_events = cold_events.append(pd.Index(["Event-51"]))

# the index of events started in the warm season
warm_events = list(set(percolation_events.index) - set(cold_events))

# which characteristics to consider:
# volume, duration, ave intensity, ave peak intensity, ave water/percolation ratio
data_warm_cold_stats = percolation_events.copy()
data_warm_cold_stats["group"] = ""
data_warm_cold_stats.loc[cold_events, "group"] = "Cold Season"
data_warm_cold_stats.loc[warm_events, "group"] = "Warm Season"

data_warm_cold_stats["water/perc ratio"] = data_warm_cold_stats["SVS_water_input_mm"] / data_warm_cold_stats["percolation_volume_mm"]
data_warm_cold_stats["percolation_duration_days"] = data_warm_cold_stats["percolation_duration_h"] / 24.0

# data_warm_cold_stats.groupby("group").agg({
#     "percolation_volume_mm": ["mean", "median", "std"],
#     "percolation_duration_days": ["mean", "median", "std"],
#     "percolation_max_intensity_mmh": ["mean", "median", "std"],
#     "percolation_ave_intensity_mmh": ["mean", "median", "std"],
#     "water/perc ratio": ["mean", "median", "std"],
# })

# plot _________________________________________________________________________
# constants
SAVE_PATH = paths["fig_dir"] / "Manuscript1_WarmColdEvents_boxplots.pdf"
FIGSIZE = (22, 18)
FONTSIZE = 45
FONT_LEGEND = FONTSIZE - 12
BWIDHT = 0.5
LWIDTH = 5
MSIZE = 13

# plot
fig, axes = plt.subplots(2, 2, figsize=FIGSIZE, sharex=True)

sns.boxplot(
    x="group", y="percolation_volume_mm", data=data_warm_cold_stats,
    ax=axes[0, 0], palette=[GOLD_BROWN, LIGHT_BLUE],
    showfliers=True, dodge=False, width=BWIDHT, linewidth=LWIDTH,
    fliersize=MSIZE,
    hue="group", legend=False, linecolor="black"

)

sns.boxplot(
    x="group", y="percolation_duration_days", data=data_warm_cold_stats,
    ax=axes[0, 1], palette=[GOLD_BROWN, LIGHT_BLUE],
    showfliers=True, dodge=False, width=BWIDHT, linewidth=LWIDTH,
    fliersize=MSIZE,
    hue="group", legend=False, linecolor="black"
)

sns.boxplot(
    x="group", y="percolation_max_intensity_mmh", data=data_warm_cold_stats,
    ax=axes[1, 0], palette=[GOLD_BROWN, LIGHT_BLUE], linewidth=LWIDTH,
    showfliers=True, dodge=False, width=BWIDHT,
    fliersize=MSIZE,
    hue="group", legend=False, linecolor="black"
)

sns.boxplot(
    x="group", y="percolation_ave_intensity_mmh", data=data_warm_cold_stats,
    ax=axes[1, 1], palette=[GOLD_BROWN, LIGHT_BLUE], linewidth=LWIDTH,
    showfliers=False, dodge=False, width=BWIDHT, fliersize=MSIZE,
    hue="group", legend=False, linecolor="black"
)

# set labels
axes[0, 0].set_ylabel("Volume (mm)", fontsize=FONTSIZE)
axes[0, 1].set_ylabel("Duration (days)", fontsize=FONTSIZE)
axes[1, 0].set_ylabel(r"Peak Intensity (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
axes[1, 1].set_ylabel(r"Average Intensity (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)

# set xlabel
for ax_ in axes.flatten():
    ax_.set_xlabel("")

    # font
    ax_.tick_params(axis='both', which='major', labelsize=FONTSIZE)

    # only 5 ticks on y axis
    ax_.locator_params(axis='y', nbins=5)

# decrease margin between subplots
plt.subplots_adjust(wspace=0.2, hspace=0.1)

# set (a) and (b) labels on top of each subplot
for i, ax_ in enumerate(axes.flatten()):
    ax_.text(
        0.5, 1.01, f"({chr(97 + i)})", transform=ax_.transAxes,
        size=FONTSIZE, weight='bold'
    )


# save
plt.tight_layout()
plt.savefig(SAVE_PATH, bbox_inches='tight', dpi=2000)
plt.show()

# Correlation analysis

In [ ]:
# Cold-season events ___________________________________________________________
PERCOLATION_EVENT_VOLUME_LABEL = "percolation_volume_mm"
PERCOLATION_EVENT_DURATION_LABEL = "percolation_duration_h"
DEP_VARS = [
    "SVS_water_input_mm",
    "SHII",
    "surface75mm_antecedent_SoilMoisture_percent",
    "surface75mm_antecedent_SoilTemp_celsius",
]


# Data preparation
# snow_events = percolation_events[percolation_events["ave_snowdepth_cm"] > 1].index
cold_events = percolation_events.loc[
    (percolation_events["ave_snowdepth_prewindow_cm"] > 1) |
    (percolation_events["AirTemperature_prewindow_celsius"] < 0)
].index

# add Event-51
cold_events = cold_events.append(pd.Index(["Event-51"]))

print(
    f"Number of cold-season events: {len(cold_events)} "
    f"({len(cold_events) / len(percolation_events) * 100:.1f}%)"
)

data_cold_events = percolation_events.loc[cold_events, :].copy()

# Initialize DataFrames
cold_pearson_r_percvolume_df = uf.create_correlation_dataframe(DEP_VARS)
cold_partial_r_percvolume_df = uf.create_correlation_dataframe(DEP_VARS)

# set dtype for the columns as float
data_cold_events[DEP_VARS] = data_cold_events[DEP_VARS].fillna(np.nan)
data_cold_events = data_cold_events.astype({col: "float64" for col in DEP_VARS})

# Calculate correlations
for col in DEP_VARS:
    cold_pearson_r_percvolume_df.loc["Coeff", col], cold_pearson_r_percvolume_df.loc["p-value", col] = \
        uf.calculate_pearson_correlation(data_cold_events, PERCOLATION_EVENT_VOLUME_LABEL, col)

    cold_partial_r_percvolume_df.loc["Coeff", col], cold_partial_r_percvolume_df.loc["p-value", col] = \
        uf.calculate_partial_correlation(data_cold_events, PERCOLATION_EVENT_VOLUME_LABEL, col, [ivar for ivar in DEP_VARS if ivar != col])

# pd.concat([cold_pearson_r_percvolume_df, cold_partial_r_percvolume_df], axis=0)

# Warm-season events ___________________________________________________________
warm_events = list(set(percolation_events.index) - set(cold_events))
print(
    f"Number of warm-season events: {len(warm_events)} "
    f"({len(warm_events) / len(percolation_events) * 100:.1f}%)"
)

data_warm_events = percolation_events.loc[warm_events, :].copy()

data_warm_events[DEP_VARS] = data_warm_events[DEP_VARS].fillna(np.nan)
data_warm_events = data_warm_events.astype({col: "float64" for col in DEP_VARS})

# Initialize DataFrames
warm_pearson_r_percvolume_df = uf.create_correlation_dataframe(DEP_VARS)
warm_partial_r_percvolume_df = uf.create_correlation_dataframe(DEP_VARS)

# Calculate correlations
for col in DEP_VARS:
    warm_pearson_r_percvolume_df.loc["Coeff", col], warm_pearson_r_percvolume_df.loc["p-value", col] = \
        uf.calculate_pearson_correlation(data_warm_events, PERCOLATION_EVENT_VOLUME_LABEL, col)

    warm_partial_r_percvolume_df.loc["Coeff", col], warm_partial_r_percvolume_df.loc["p-value", col] = \
        uf.calculate_partial_correlation(data_warm_events, PERCOLATION_EVENT_VOLUME_LABEL, col, [ivar for ivar in DEP_VARS if ivar != col])

# pd.concat([warm_pearson_r_percvolume_df, warm_partial_r_percvolume_df], axis=0)

# plot _________________________________________________________________________
SAVE_PATH = paths["fig_dir"] / "Manuscript1_correlation_BothGroupEvents.pdf"
FIGSIZE = (14, 14)
FONTSIZE = 40
FONT_LEGEND = FONTSIZE - 12

HALF_WIDTH = 0.15
SHORT_NAMES = [
    "Input Water", "Intensity Index",
    "Soil Moisture",
    "Soil Temperature",
]



# Create figure
fig, ax = plt.subplots(nrows=2, ncols=1, figsize=FIGSIZE, sharex=True)

# Datasets (Assumed to be defined elsewhere in your code)
# pearson_r_percvolume_df, partial_r_percvolume_df
# pearson_r_percduration_df, partial_r_percduration_df

dataframes = [(warm_pearson_r_percvolume_df, warm_partial_r_percvolume_df), (cold_pearson_r_percvolume_df, cold_partial_r_percvolume_df)]
titles = ["Warm-Season Eevnts", "Cold-Season Events"]

for i, (pearson_df, partial_df) in enumerate(dataframes):
    for icol, col in enumerate(pearson_df.columns):
        uf.plot_rectangle_and_text(
            ax[i], icol, pearson_df.loc["Coeff", col],
            pearson_df.loc["p-value", col], 0.5, GOLD_BROWN, HALF_WIDTH,
            FONT_LEGEND, lwdith=3.5,
        )
        uf.plot_rectangle_and_text(
            ax[i], icol, partial_df.loc["Coeff", col],
            partial_df.loc["p-value", col], -0.5, LIGHT_BLUE, HALF_WIDTH,
            FONT_LEGEND, lwdith=3.5,
        )

    ax[i].set_ylim(-1, 1)
    ax[i].set_aspect('equal')
    ax[i].set_yticks([0.5, -0.5])
    ax[i].set_yticklabels(["Total\n Corr.", "Partial\n Corr."], fontsize=FONT_LEGEND)
    # ax[i].set_title(f"({chr(97 + i)})", fontsize=FONTSIZE)
    ax[i].text(0.5, 1.04, f"({chr(97 + i)}) {titles[i]}", fontsize=FONT_LEGEND, ha="center", va="center", transform=ax[i].transAxes)


ax[1].set_xticks(np.arange(len(SHORT_NAMES)))
ax[1].set_xticklabels(SHORT_NAMES, fontsize=FONT_LEGEND - 3, rotation=0, ha="center")

plt.tight_layout()
plt.savefig(SAVE_PATH, dpi=2000, bbox_inches="tight")
plt.show()

# Plot Volume / Duration Distributions

In [ ]:
SAVE_PATH = paths["fig_dir"] / "Manuscript1_events_vol_duration_hist.pdf"
fgsize = (25, 12)
fontsize = 45
fontsize_legend = fontsize - 6

# copy the dataframe and convert duration to days
data_events = percolation_events.copy()
data_events["percolation_duration_d"] = data_events["percolation_duration_h"] / 24.0

fig, ax = plt.subplots(1, 2, figsize=fgsize, sharey=True)

# plot histogram of event volume
sns.histplot(
    x="percolation_volume_mm",
    data=data_events,
    ax=ax[0],
    color=LIGHT_BLUE,
    edgecolor="black",
    bins=5,
    kde=True,
    stat="count",
    line_kws={"lw": 4.5, "ls": "--", "color": "black"},
    alpha=0.7,
)
kde_line = ax[0].lines[0]
kde_line.set_color("black")


# plot histogram of event duration
sns.histplot(
    x="percolation_duration_d",
    data=data_events,
    ax=ax[1],
    color=GOLD_BROWN,
    edgecolor="black",
    bins=5,
    kde=True,
    stat="count",
    line_kws={"lw": 4.5, "ls": "--", "color": "black"},
    alpha=0.7,
)
kde_line = ax[1].lines[0]
kde_line.set_color("black")


# set xlabels
ax[0].set_xlabel("Percolation Volume (mm)", fontsize=fontsize)
ax[1].set_xlabel("Percolation Duration (day)", fontsize=fontsize)

# set ylabels
ax[0].set_ylabel("Frequency", fontsize=fontsize)

# set titles
ax[0].set_title("(a)", loc="center", fontsize=fontsize)
ax[1].set_title("(b)", loc="center", fontsize=fontsize)

# set tick params
ax[0].tick_params(axis="both", which="major", labelsize=fontsize)
ax[1].tick_params(axis="both", which="major", labelsize=fontsize)


# for each figure, annotate the dist stats: mean, std, kurtosis, skewness
for xnr, xname in enumerate(["percolation_volume_mm", "percolation_duration_d"]):
    ax[xnr].text(
        0.68,
        0.97,
        f"Mean: {data_events.loc[:, xname].mean():.1f}\n"
        f"Median: {data_events.loc[:, xname].median():.1f}\n"
        f"Std: {data_events.loc[:, xname].std():.1f}\n",
        transform=ax[xnr].transAxes,
        fontsize=fontsize_legend,
        verticalalignment="top",
        bbox=dict(boxstyle="round", facecolor="white", alpha=0.5),
    )

# save fig
plt.tight_layout()
plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=2000)
plt.show()

# General stats

In [ ]:
# copy the percolation events dataframe to a new dataframe
data_events = percolation_events.copy()

# add duration in days
data_events["percolation_duration_d"] = data_events["percolation_duration_h"] / 24

# print info on the event with max volume
max_volume_event = data_events.loc[data_events["percolation_volume_mm"].idxmax()]
uf.print_event_info(max_volume_event, "Event with max volume")

# print info on the event with min volume
min_volume_event = data_events.loc[data_events["percolation_volume_mm"].idxmin()]
uf.print_event_info(min_volume_event, "Event with min volume")

# print info on the event with max duration
max_duration_event = data_events.loc[data_events["percolation_duration_h"].idxmax()]
uf.print_event_info(max_duration_event, "Event with max duration")

# print info on the event with min duration
min_duration_event = data_events.loc[data_events["percolation_duration_h"].idxmin()]
uf.print_event_info(min_duration_event, "Event with min duration")

# min, ave, median, max volume of events
print(
    f"Min, ave, median, max volume of events: {data_events['percolation_volume_mm'].min()}, "
    f"{data_events['percolation_volume_mm'].mean()}, {data_events['percolation_volume_mm'].median()}, "
    f"{data_events['percolation_volume_mm'].max()}"
)

# cv of volume of events
print(
    f"CV of volume of events: {data_events['percolation_volume_mm'].std() / data_events['percolation_volume_mm'].mean()}"
)

# min, ave, median, max duration of events, days
print(
    f"Min, ave, median, max duration of events: {data_events['percolation_duration_d'].min()}, "
    f"{data_events['percolation_duration_d'].mean()}, {data_events['percolation_duration_d'].median()}, "
    f"{data_events['percolation_duration_d'].max()}"
)

# standard deviation of duration of events, days
print(
    f"Standard deviation of duration of events: {data_events['percolation_duration_d'].std()}"
)

# Seasonal stats

In [ ]:
data_events_cp = data_events.copy()

# add a `season` column, fill according to percolation start date
month_to_season = {
    1: "Winter", 2: "Winter", 12: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Fall", 10: "Fall", 11: "Fall",

}
data_events_cp["season"] = ""
for event_id, event in data_events_cp.iterrows():
    month = event["percolation_start_dt"].month
    data_events_cp.loc[event_id, "season"] = month_to_season[month]

# add a snowmelt column, diff between rainfall and input water
data_events_cp["snowmelt"] = -data_events_cp["rainfall_volume_mm"] + data_events_cp["SVS_water_input_mm"]

# only consider events started before June 2021
# data_events_cp = data_events_cp.loc[data_events_cp["percolation_start_dt"] < "2021-06-01"]

seasonal_stats = data_events_cp.groupby("season").agg({
    "percolation_volume_mm": ["mean", "median", "std"],
    "percolation_duration_h": ["mean", "median", "std"],
    "percolation_max_intensity_mmh": ["mean", "median", "std"],
    "percolation_ave_intensity_mmh": ["mean", "median", "std"],
    "snowmelt": ["mean", "median", "std"],
    "rainfall_volume_mm": ["mean", "median", "std"],
})

# re order index, Spring, Summer, Fall, Winter
seasonal_stats = seasonal_stats.reindex(["Spring", "Summer", "Fall", "Winter"])


# Reshape the data for plotting
plot_data = pd.DataFrame({
    "Percolation Volume (mean)": seasonal_stats["percolation_volume_mm"]["mean"],
    "Rainfall Volume (mean)": seasonal_stats["rainfall_volume_mm"]["mean"],
    "Snowmelt (mean)": seasonal_stats["snowmelt"]["mean"],
})
plot_data = plot_data.reindex(["Spring", "Summer", "Fall", "Winter"])  # Ensure the order of seasons


# constants
SAVE_PATH = paths["fig_dir"] / "Manuscript1_barplot_seasonal_events.pdf"
FIGSIZE = (25, 12)
FONTSIZE = 45
FONT_LEGEND = FONTSIZE - 6
BAR_WIDTH = 0.3
BAR_ALPHA = 0.7

fig, ax = plt.subplots(figsize=FIGSIZE)

# Bar width and positions
x = np.arange(len(plot_data))  # Positions of the bars on the x-axis

# Plot bars
bars1 = ax.bar(x - BAR_WIDTH/2, plot_data["Percolation Volume (mean)"], BAR_WIDTH,
               label="Percolation", color=LIGHT_BLUE, edgecolor='black',
                alpha=BAR_ALPHA
               )

# Stacked bars for rain and snowmelt
bars2 = ax.bar(x + BAR_WIDTH/2, plot_data["Rainfall Volume (mean)"], BAR_WIDTH,
               label="Rain",
               edgecolor='black', hatch='', color=GOLD_BROWN,
                alpha=BAR_ALPHA

            )
bars3 = ax.bar(x + BAR_WIDTH/2, plot_data["Snowmelt (mean)"], BAR_WIDTH,
               bottom=plot_data["Rainfall Volume (mean)"], label="Snowmelt",
               edgecolor='black', alpha=BAR_ALPHA,
               hatch='o', color=LIGHT_GREY,
            )

# Add labels, title, and legend
ax.set_xlabel("", fontsize=FONTSIZE)
ax.set_ylabel("Volume (mm)", fontsize=FONTSIZE)
ax.set_xticks(x)
ax.set_xticklabels(plot_data.index, fontsize=FONTSIZE)
ax.legend(fontsize=FONT_LEGEND, frameon=False, loc="upper left", ncols=3)

ax.set_ylim(0, 119)

# Set font size for ticks
ax.tick_params(axis='both', which='major', labelsize=FONTSIZE)

# add (c)
ax.text(
    0.5, 1.01, "(c)", transform=ax.transAxes,
    size=FONTSIZE, weight='bold'
)


# Save the figure
plt.tight_layout()
plt.savefig(SAVE_PATH, dpi=2000, bbox_inches="tight")
plt.show()

# Plot Event-9

In [ ]:
EVENT_ID = "Event-9"
SAVE_PATH = paths["fig_dir"] / f"Manuscript1_{EVENT_ID}.pdf"
FIGSIZE = (19, 18)
FONTSIZE = 37
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 4
BWIDHT = 0.04

percolation_label = "Enclosure3_Lysimeter4_percolation_mmh"
soil_moist_sensor_label = "L4_75mm_ext_Moisture_percent"
soil_temp_sensor_label = "L4_75mm_ext_Temperature_degc"

fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

t0 = percolation_events.loc[EVENT_ID]["percolation_start_dt"]
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[EVENT_ID]["percolation_end_dt"]

# mofidy t00
t00 += pd.Timedelta(days=2)

t00_date = t00.date().strftime("%Y-%m-%d")
t1_date = t1.date().strftime("%Y-%m-%d")


# hourly rainfall, water input, and snow depth
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1],
    width=BWIDHT,
    color=LIGHT_BLUE,
    label="Rainfall",
    alpha=0.75,
)
ax[0].bar(
    svs_ens_ave.loc[t00:t1].index,
    svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"],
    width=BWIDHT,
    color=LIGHT_GREY,
    label="Simulated Input Water",
    alpha=0.75,
)

tax0 = ax[0].twinx()
tax0.plot(
    data_meteo_daily.loc[data_meteo_daily["Variable"] == "Snow depth", "Reading"].loc[t00_date:t1_date].index,
    data_meteo_daily.loc[data_meteo_daily["Variable"] == "Snow depth", "Reading"].loc[t00_date:t1_date],
    color="black",
    linestyle="-",
    linewidth=LINEWIDTH,
    label="Snow depth",
)

# surface soil moisture and temperature
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil_moist_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil_moist_sensor_label, "Reading"].loc[t00:t1],
    color=LIGHT_BLUE,
    linewidth=LINEWIDTH,
    label="Soil moisture",
)

tax1 = ax[1].twinx()
tax1.plot(
    data_soil_temp.loc[data_soil_temp["Unique_label_"] == soil_temp_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_temp.loc[data_soil_temp["Unique_label_"] == soil_temp_sensor_label, "Reading"].loc[t00:t1],
    color=GOLD_BROWN,
    linewidth=LINEWIDTH,
    label="Soil temperature",
    linestyle="-.",
)
# limit x from 2018-12-20 to ...
ax[0].set_xlim([pd.Timestamp("2018-12-20"), t1])

# percolation
ax[2].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t00:t1].index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t00:t1],
    width=BWIDHT,
    color=LIGHT_BLUE,
    label="Percolation (L4)",
)

tax2= ax[2].twinx()
tax2.plot(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Air temperature", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Air temperature", "Reading"].loc[t00:t1],
    color=GOLD_BROWN,
    linestyle="-",
    linewidth=LINEWIDTH,
    label="Air temperature",
)


# fonts and labels
ax[0].set_ylabel(r"Input Water (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
tax0.set_ylabel("Snow Depth (cm)", fontsize=FONTSIZE)
ax[1].set_ylabel("Soil Moisture (%)", fontsize=FONTSIZE)
tax1.set_ylabel(r"Soil Temperature ($^\circ$C)", fontsize=FONTSIZE)
ax[2].set_ylabel(r"Percolation (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
tax2.set_ylabel(r"Air Temperature ($^\circ$C)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
tax0.tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
tax1.tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)
tax2.tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
ax[0].legend(loc=(0.00, 0.01), fontsize=FONT_LEGEND, frameon=False)
tax0.legend(loc=(0.75, 0.85), fontsize=FONT_LEGEND, frameon=False)

ax[1].legend(loc=(0.00, 0.85), fontsize=FONT_LEGEND, frameon=False)
tax1.legend(loc=(0.27, 0.85), fontsize=FONT_LEGEND, frameon=False)

ax[2].legend(loc=(0.00, 0.85), fontsize=FONT_LEGEND, frameon=False)
tax2.legend(loc=(0.35, 0.85), fontsize=FONT_LEGEND, frameon=False)

# increase tax1 lim
tax1.set_ylim([-1.99, 1.99])

# zero degree h lines
tax1.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)
tax2.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

# increase pad between xtick and ytick for ax2
ax[2].tick_params(axis="x", which="major", pad=15)

# yticks on tax0 only 0 5 10
tax0.set_yticks([0, 5, 10])

# yticks on ax0 only 0 3 6
ax[0].set_yticks([0, 3, 6])

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)

# only t00, and t1 have xticks
# set t00 to 2018-12-20 00, t1 to 2018-12-21 00, t0 to 2018-12-23 00
t00 = pd.Timestamp("2018-12-20 04:00:00", tz="UTC")
t0 = pd.Timestamp("2018-12-21 04:00:00", tz="UTC")
t11 = pd.Timestamp("2018-12-22 04:00:00", tz="UTC")
t1 = pd.Timestamp("2018-12-23 04:00:00", tz="UTC")
ax[2].set_xticks([t00, t0, t11, t1])

# change tick labels: year only for first tick, the rest: month-day
ax[2].set_xticklabels(
    [
        # "Dec 20 (2018)",
        "2018-12-20",
        "21", "22", "23"
    ]
)

ax[2].set_ylim(0, 13)


plt.tight_layout()
plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=2000)
plt.show()

# Plot Event-47

In [ ]:
SAVE_PATH = paths["fig_dir"] / f"Manuscript1_Event47.pdf"
FIGSIZE = (19, 24)
FONTSIZE = 39
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 5
BWIDHT = 0.2

suc1850_l2_label = "L2_1850mm_int_Suction_kPa"
suc1750_l2_label = "L2__1750mm_int_Suction_kPa"
suc600_l2_label = "L2_600mm_int_Suction_kPa"
moist75_l2_label = "L2_75mm_ext_Moisture_percent"
moist225_l2_label = "L2_225mm_ext_Moisture_percent"
moist1850_l2_label = "L2_1850mm_int_Moisture_percent"
moist1750_l2_label = "L2_1750mm_int_Moisture_percent"
moist600_l2_label = "L2_600mm_int_Moisture_percent"

suc1850_l1_label = "L1_1850mm_int_Suction_kPa"
suc1750_l1_label = "L1_1750mm_int_Suction_kPa"
moist75_l1_label = "L1_75mm_ext_Moisture_percent"
moist225_l1_label = "L1_225mm_ext_Moisture_percent"

suc1850_l4_label = "L4_1850mm_int_Suction_kPa"
suc1750_l4_label = "L4_1750mm_int_Suction_kPa"
suc280_l4_label = "L4_280mm_int_Suction_kPa"
moist75_l4_label = "L4_75mm_ext_Moisture_percent"
moist225_l4_label = "L4_225mm_ext_Moisture_percent"
moist280_l4_label = "L4_280mm_int_Moisture_percent"
moist1850_l4_label = "L4_1850mm_int_Moisture_percent"
moist1750_l4_label = "L4_1750mm_int_Moisture_percent"

suc1850_l3_label = "L3_1850mm_int_Suction_kPa"
suc1750_l3_label = "L3_1750mm_int_Suction_kPa"
suc600_l3_label = "L3_600mm_int_Suction_kPa"
moist75_l3_label = "L3_75mm_ext_Moisture_percent"
moist225_l3_label = "L3_225mm_ext_Moisture_percent"
moist600_l3_label = "L3_600mm_int_Moisture_percent"
moist1850_l3_label = "L3_1850mm_int_Moisture_percent"
moist1750_l3_label = "L3_1750mm_int_Moisture_percent"

fig, ax = plt.subplots(4, 1, figsize=FIGSIZE, sharex=True)

t0 = pd.Timestamp("2020-09-30 20:00:00+00:00")
t1 = pd.Timestamp("2020-10-22 03:00:00+00:00")
t00 = t0 - pd.Timedelta(days=4)


# hourly rainfall
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1],
    color=LIGHT_BLUE,
    label="Rainfall", width=BWIDHT,
)

# moist 75
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l2_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    linewidth=LINEWIDTH,
    label="75 mm",
)


ax[2].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l2_label, "Reading"].loc[t00:t1].mul(100),
    color=GOLD_BROWN,
    linestyle="-", linewidth=LINEWIDTH, label="225 mm",
)
ax[2].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist600_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist600_l2_label, "Reading"].loc[t00:t1].mul(100),
    color=DARK_PURPLE,
    linestyle="-.", linewidth=LINEWIDTH, label="600 mm",
)


ax[3].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, "Reading"].loc[t0:t1].index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, "Reading"].loc[t0:t1],
    color=LIGHT_BLUE,
    label="L2", width=BWIDHT,
)

# fonts and labels
ax[0].set_ylabel(r"Rainfall (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
ax[1].set_ylabel(r"Soil Moisture (%)", fontsize=FONTSIZE)
ax[2].set_ylabel(r"Soil Moisture (%)", fontsize=FONTSIZE)
ax[3].set_ylabel(r"Percolation (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[3].tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
# ax[0].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82))
ax[1].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=3)
ax[2].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)
ax[3].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82))

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)", "(d)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)


# xticks only: 2019-05-02, 2019-05-06, 2019-05-15
ax[0].set_xticks(
    [
        pd.Timestamp("2020-09-30 20:00:00-0000"),
        # pd.Timestamp("2019-05-06 21:00:00-0000"),
        pd.Timestamp("2020-10-22 03:00:00+0000"),
    ]
)

# tick labels: year only for first tick, the rest: month-day
ax[0].set_xticklabels(
    [
        "2020-09-26", "10-22",
    ]
)


ax[2].tick_params(axis="x", which="major", pad=15)

ax[1].set_ylim(10, 39.9)
ax[2].set_ylim(10, 22)

plt.tight_layout()
plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=2000)
plt.show()

# Plot Event-34

In [ ]:
EVENT_ID = "Event-34"
SAVE_PATH = paths["fig_dir"] / f"Manuscript1_{EVENT_ID}.pdf"
FIGSIZE = (19, 24)
FONTSIZE = 39
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 5
BWIDHT = 0.04

percolation_label = "Enclosure3_Lysimeter4_percolation_mmh"
soil_moist_sensor_label = "L4_75mm_ext_Moisture_percent"
soil_temp_sensor_label = "L4_75mm_ext_Temperature_degc"

soil225mm_moist_sensor_label = "L4_225mm_ext_Moisture_percent"
soil280mm_moist_sensor_label = "L4_280mm_int_Moisture_percent"
soil1750mm_moist_sensor_label = "L4_1750mm_int_Moisture_percent"
soil1850mm_moist_sensor_label = "L4_1850mm_int_Moisture_percent"

fig, ax = plt.subplots(4, 1, figsize=FIGSIZE, sharex=True)

t0 = percolation_events.loc[EVENT_ID]["percolation_start_dt"]
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[EVENT_ID]["percolation_end_dt"]


# set t00 to '2020-01-09 19:00:00-0500'
# set t1 to '2020-01-12 19:00:00-0500'
# this adjustment is needed regarding the daily snow depth data
t00 = pd.Timestamp('2020-01-10 00:00:00-0500')
t1 = pd.Timestamp('2020-01-14 00:00:00-0500')

t00_date = t00.date().strftime("%Y-%m-%d")
t1_date = t1.date().strftime("%Y-%m-%d")

# hourly rainfall, water input, and snow depth
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1],
    width=BWIDHT,
    color=LIGHT_BLUE,
    label="Rainfall",
    alpha=0.75,
)
ax[0].bar(
    svs_ens_ave.loc[t00:t1].index,
    svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"],
    width=BWIDHT,
    color=LIGHT_GREY,
    label="Simulated Input Water",
    alpha=0.75,
)

tax0 = ax[0].twinx()
tax0.plot(
    data_meteo_daily.loc[data_meteo_daily["Variable"] == "Snow depth", "Reading"].loc[t00_date:t1_date].index,
    data_meteo_daily.loc[data_meteo_daily["Variable"] == "Snow depth", "Reading"].loc[t00_date:t1_date],
    color="black",
    linestyle="-",
    linewidth=LINEWIDTH,
    label="Snow depth",
)

# surface soil moisture and temperature
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil_moist_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil_moist_sensor_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    linestyle="-",
    linewidth=LINEWIDTH,
    label="Soil moisture (75 mm)",
)

tax1 = ax[1].twinx()
tax1.plot(
    data_soil_temp.loc[data_soil_temp["Unique_label_"] == soil_temp_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_temp.loc[data_soil_temp["Unique_label_"] == soil_temp_sensor_label, "Reading"].loc[t00:t1],
    color=GOLD_BROWN,
    linewidth=LINEWIDTH, linestyle="-.",
    label="Soil temperature (75 mm)",
)

# the three soil sensors on ax2
ax[2].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil225mm_moist_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil225mm_moist_sensor_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    linestyle="-",
    linewidth=LINEWIDTH,
    label="225 mm",
)
ax[2].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil280mm_moist_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil280mm_moist_sensor_label, "Reading"].loc[t00:t1].mul(100),
    color=DARK_PURPLE,
    linestyle="-.",
    linewidth=LINEWIDTH,
    label="280 mm",
)

# percolation
ax[3].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t00:t1].index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t00:t1],
    width=BWIDHT,
    color=LIGHT_BLUE,
    label="Percolation (L4)",
)

tax3= ax[3].twinx()
tax3.plot(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Air temperature", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Air temperature", "Reading"].loc[t00:t1],
    color=GOLD_BROWN,
    linestyle="-",
    linewidth=LINEWIDTH,
    label="Air temperature",
)


# fonts and labels
ax[0].set_ylabel(r"Input Water (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
tax0.set_ylabel("Snow Depth (cm)", fontsize=FONTSIZE)
ax[1].set_ylabel("Soil Moisture (%)", fontsize=FONTSIZE)
tax1.set_ylabel(r"Soil Temperature ($^\circ$C)", fontsize=FONTSIZE)
ax[2].set_ylabel("Soil Moisture (%)", fontsize=FONTSIZE)
ax[3].set_ylabel(r"Percolation (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
tax3.set_ylabel(r"  Air Temperature ($^\circ$C)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
tax0.tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
tax1.tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[3].tick_params(axis="both", which="major", labelsize=FONTSIZE)
tax3.tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
ax[0].legend(loc=(0.00, 0.01), fontsize=FONT_LEGEND, frameon=False)
tax0.legend(loc=(0.72, 0.85), fontsize=FONT_LEGEND, frameon=False)

ax[1].legend(loc=(0.00, 0.85), fontsize=FONT_LEGEND, frameon=False)
tax1.legend(loc=(0.00, 0.69), fontsize=FONT_LEGEND, frameon=False)

ax[3].legend(loc=(0.00, 0.85), fontsize=FONT_LEGEND, frameon=False)
tax3.legend(loc=(0.68, 0.85), fontsize=FONT_LEGEND, frameon=False)

# horizontal legend for ax2
ax[2].legend(
    loc="upper left",
    ncol=2,
    fontsize=FONT_LEGEND,
    frameon=False,
)


# zero degree h lines
tax1.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)
tax3.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

# increase pad between xtick and ytick for ax2
ax[3].tick_params(axis="x", which="major", pad=15)

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)", "(d)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)

# axis adjustments
tax1.set_ylim(-2, 2)
tax1.set_yticks([-1, 0, 1])

tax3.set_ylim(-15, 15)

ax[2].set_ylim(15, 28)

# only 0, 5, 10 on tax0 y
tax0.set_yticks([0, 5, 10])

# xticks only: 2020-01-09 00:00 and increemnt by 1 day four times
ax[0].set_xticks(
    [
        pd.Timestamp("2020-01-10 00:00:00-0500"),
        pd.Timestamp("2020-01-11 00:00:00-0500"),
        pd.Timestamp("2020-01-12 00:00:00-0500"),
        pd.Timestamp("2020-01-13 00:00:00-0500"),
        pd.Timestamp("2020-01-14 00:00:00-0500"),
    ]
)

# tick labels: year only for first tick, the rest: month-day
ax[0].set_xticklabels(
    [
        # "Jan 10 (2020)",
        "2020-01-10",
        "11", "12", "13", "14",
    ]
)

ax[0].yaxis.labelpad = 20
ax[1].yaxis.labelpad = 20
ax[3].yaxis.labelpad = 25


plt.tight_layout()
plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=2000)
plt.show()

# Event 43 (summer 2020)

In [ ]:
EVENT_ID = "Event-43"
SAVE_PATH = paths["fig_dir"] / f"Manuscript1_{EVENT_ID}.pdf"
FIGSIZE = (19, 24)
FONTSIZE = 39
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 5
BWIDHT = 0.2

lys_nr = 2

percolation_label = "Enclosure1_Lysimeter1_percolation_mmh"
soil_moist_sensor_label = f"L{lys_nr}_75mm_ext_Moisture_percent"
soil_temp_sensor_label = f"L{lys_nr}_75mm_ext_Temperature_degc"

soil225mm_moist_sensor_label = f"L{lys_nr}_225mm_ext_Moisture_percent"
soil600mm_moist_sensor_label = f"L{lys_nr}_600mm_int_Moisture_percent"
soil1750mm_moist_sensor_label = f"L{lys_nr}_1750mm_int_Moisture_percent"
soil1850mm_moist_sensor_label = f"L{lys_nr}_1850mm_int_Moisture_percent"

soil_drain_suction_label = f"L{lys_nr}_drain_Suction_kPa"
soil_1850mm_suction_label = f"L{lys_nr}_1850mm_int_Suction_kPa"
soil_1750mm_suction_label = f"L{lys_nr}_1750mm_int_Suction_kPa"
soil_1750mm_suction_label = "L2__1750mm_int_Suction_kPa"




fig, ax = plt.subplots(4, 1, figsize=FIGSIZE, sharex=True)

t0 = percolation_events.loc[EVENT_ID]["percolation_start_dt"]
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[EVENT_ID]["percolation_end_dt"]

# mofidy t00
t00 -= pd.Timedelta(days=5)
t00_date = t00.date().strftime("%Y-%m-%d")
t1_date = t1.date().strftime("%Y-%m-%d")

# hourly rainfall, water input, and snow depth
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1],
    width=BWIDHT,
    color=LIGHT_BLUE,
    label="Rainfall",
    alpha=0.75,
)

# surface soil moisture and temperature
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil_moist_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil_moist_sensor_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    linestyle="-",
    linewidth=LINEWIDTH,
    label="75 mm",
)

# the three soil sensors on ax2
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil225mm_moist_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil225mm_moist_sensor_label, "Reading"].loc[t00:t1].mul(100),
    color='k',
    linestyle=":",
    linewidth=LINEWIDTH,
    label="225 mm",
)
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil600mm_moist_sensor_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == soil600mm_moist_sensor_label, "Reading"].loc[t00:t1].mul(100),
    color=DARK_PURPLE,
    linestyle="-.",
    linewidth=LINEWIDTH,
    label="600 mm",
)

ax[2].plot(
    data_soil_suction.loc[data_soil_suction["Unique_label_"] == soil_1850mm_suction_label, "Reading"].loc[t00:t1].index,
    data_soil_suction.loc[data_soil_suction["Unique_label_"] == soil_1850mm_suction_label, "Reading"].loc[t00:t1],
    color=GOLD_BROWN,
    linestyle="-",
    linewidth=LINEWIDTH,
    label="1850 mm",
)


# percolation
ax[3].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == lys_nr, "Reading"].loc[t00:t1].index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == lys_nr, "Reading"].loc[t00:t1],
    width=BWIDHT,
    color=LIGHT_BLUE,
    label="L2",
)


# fonts and labels
ax[0].set_ylabel(r"Rainfall (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
ax[1].set_ylabel("Soil Moisture (%)", fontsize=FONTSIZE)
# tax1.set_ylabel(r"Soil Temperature ($^\circ$C)", fontsize=FONTSIZE)
ax[2].set_ylabel("Suction (kPa)", fontsize=FONTSIZE)
ax[3].set_ylabel(r"Percolation (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
# tax1.tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[3].tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
ax[0].legend(loc=(0.0, 0.82), fontsize=FONT_LEGEND, frameon=False)
ax[1].legend(loc=(0.00, 0.82), fontsize=FONT_LEGEND, frameon=False, ncol=3)
ax[2].legend(loc=(0.00, 0.82), fontsize=FONT_LEGEND, frameon=False)
ax[3].legend(loc=(0.00, 0.82), fontsize=FONT_LEGEND, frameon=False)

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)", "(d)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)

# increase lims
ax[0].set_ylim(0, 40)
ax[1].set_ylim(5, 40)
ax[2].set_ylim(0, 2.2)
ax[3].set_ylim(0, 0.4)

# xticks only: 2020-08-05, 2020-08-15, 2020-08-25
ax[0].set_xticks(
    [
        pd.Timestamp("2020-08-01 00:00:00-0000"),
        pd.Timestamp("2020-08-08 00:00:00-0000"),
        # pd.Timestamp("2020-08-15 00:00:00-0000"),
        pd.Timestamp("2020-08-25 00:00:00-0000"),
    ]
)

# tick labels: year only for first tick, the rest: month-day
ax[0].set_xticklabels(
    [
        "2020-08-01",
        "08",
        # "15",
        "25",
    ]
)

# increase pad between xtick and ytick for ax3
ax[3].tick_params(axis="x", which="major", pad=15)

# increase pad between ylabel for ax2
ax[2].yaxis.labelpad = 30
ax[1].yaxis.labelpad = 5

plt.tight_layout()
plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=2000)
plt.show()

# Suction at 1850 for L2 L4

In [ ]:
SAVE_PATH = paths["fig_dir"] / f"Manuscript1_2020_summer_suction_L2L4.pdf"
FIGSIZE = (19, 18)
FONTSIZE = 39
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 5
BWIDHT = 0.55

suc1850_l2_label = "L2_1850mm_int_Suction_kPa"
suc1750_l2_label = "L2__1750mm_int_Suction_kPa"
suc600_l2_label = "L2_600mm_int_Suction_kPa"
moist75_l2_label = "L2_75mm_ext_Moisture_percent"
moist225_l2_label = "L2_225mm_ext_Moisture_percent"
temp225_l2_label = "L2_225mm_ext_Temperature_degc"
temp1850_l2_label = "L2_1850mm_int_Temperature_degc"
temp1750_l2_label = "L2_1750mm_int_Temperature_degc"
moist1750_l2_label = "L2_1750mm_int_Moisture_percent"
moist1850_l2_label = "L2_1850mm_int_Moisture_percent"

suc1850_l4_label = "L4_1850mm_int_Suction_kPa"
suc1750_l4_label = "L4_1750mm_int_Suction_kPa"
suc280_l4_label = "L4_280mm_int_Suction_kPa"
moist75_l4_label = "L4_75mm_ext_Moisture_percent"
moist225_l4_label = "L4_225mm_ext_Moisture_percent"
temp225_l4_label = "L4_225mm_ext_Temperature_degc"
temp1850_l4_label = "L4_1850mm_int_Temperature_degc"
temp1750_l4_label = "L4_1750mm_int_Temperature_degc"
moist1750_l4_label = "L4_1750mm_int_Moisture_percent"
moist1850_l4_label = "L4_1850mm_int_Moisture_percent"

suc1850_l3_label = "L3_1850mm_int_Suction_kPa"
suc1750_l3_label = "L3_1750mm_int_Suction_kPa"
suc600_l3_label = "L3_600mm_int_Suction_kPa"
moist75_l3_label = "L3_75mm_ext_Moisture_percent"
moist225_l3_label = "L3_225mm_ext_Moisture_percent"
temp225_l3_label = "L3_225mm_ext_Temperature_degc"

fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

t0 = pd.Timestamp("2020-05-20 00:00:00-0000")
t1 = pd.Timestamp("2020-08-26 00:00:00-0000")

# hourly rainfall
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t0:t1].resample("1D").sum().index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t0:t1].resample("1D").sum(),
    color=LIGHT_BLUE,
    label="Rainfall", width=BWIDHT,
)


# suction
ax[1].plot(
    data_soil_suction.loc[data_soil_suction["Unique_label_"] == suc1850_l2_label, "Reading"].loc[t0:t1].resample("1D").mean(),
    color=LIGHT_BLUE,
    label="1850 mm (L2)",
    linewidth=LINEWIDTH,
)
ax[1].plot(
    data_soil_suction.loc[data_soil_suction["Unique_label_"] == suc1850_l4_label, "Reading"].loc[t0:t1].resample("1D").mean(),
    color=GOLD_BROWN,
    label="1850 mm (L4)",
    linewidth=LINEWIDTH,
    linestyle="-.",
)




ax[2].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, "Reading"].loc[t0:t1].resample("D").sum().index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, "Reading"].loc[t0:t1].resample("D").sum(),
    color=LIGHT_BLUE,
    label="L2", width=BWIDHT,
)
ax[2].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t0:t1].resample("D").sum().index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t0:t1].resample("D").sum(),
    color=GOLD_BROWN, alpha=0.8,
    label="L4", width=BWIDHT, hatch="////",
)

# fonts and labels
ax[0].set_ylabel(r"Rainfall (mm $\mathregular{d^{-1}}$)", fontsize=FONTSIZE)
ax[1].set_ylabel(r"Suction (kPa)", fontsize=FONTSIZE)
ax[2].set_ylabel(r"Percolation (mm $\mathregular{d^{-1}}$)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
ax[0].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82))
ax[1].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)
ax[2].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)


# xticks only: 2020-06-01, 2020-07-01, 2020-08-01, 2020-09-01
ax[0].set_xticks(
    [
        pd.Timestamp("2020-06-01 00:00:00-0000"),
        pd.Timestamp("2020-07-01 00:00:00-0000"),
        pd.Timestamp("2020-08-01 00:00:00-0000"),
        pd.Timestamp("2020-08-25 00:00:00-0000"),
    ]
)

# tick labels: year only for first tick, the rest: month-day
ax[0].set_xticklabels(
    [
        "2020-06-01",
        "07-01", "08-01", "08-25",
    ]
)

ax[1].yaxis.labelpad = 30
ax[2].yaxis.labelpad = 30

plt.tight_layout()
plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=2000)
plt.show()

# Any cell below is not part of the manuscript analyses

# Event x

In [ ]:
SAVE_PATH = paths["fig_dir"] / f"2020_fall_L4.pdf"
FIGSIZE = (19, 24)
FONTSIZE = 39
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 5
BWIDHT = 0.3

suc1850_l2_label = "L2_1850mm_int_Suction_kPa"
suc1750_l2_label = "L2__1750mm_int_Suction_kPa"
suc600_l2_label = "L2_600mm_int_Suction_kPa"
moist75_l2_label = "L2_75mm_ext_Moisture_percent"
moist225_l2_label = "L2_225mm_ext_Moisture_percent"

suc1850_l1_label = "L1_1850mm_int_Suction_kPa"
suc1750_l1_label = "L1_1750mm_int_Suction_kPa"
moist75_l1_label = "L1_75mm_ext_Moisture_percent"
moist225_l1_label = "L1_225mm_ext_Moisture_percent"

suc1850_l4_label = "L4_1850mm_int_Suction_kPa"
suc1750_l4_label = "L4_1750mm_int_Suction_kPa"
suc280_l4_label = "L4_280mm_int_Suction_kPa"
moist75_l4_label = "L4_75mm_ext_Moisture_percent"
moist225_l4_label = "L4_225mm_ext_Moisture_percent"
moist280_l4_label = "L4_280mm_int_Moisture_percent"
moist1850_l4_label = "L4_1850mm_int_Moisture_percent"
moist1750_l4_label = "L4_1750mm_int_Moisture_percent"

suc1850_l3_label = "L3_1850mm_int_Suction_kPa"
suc1750_l3_label = "L3_1750mm_int_Suction_kPa"
suc600_l3_label = "L3_600mm_int_Suction_kPa"

fig, ax = plt.subplots(4, 1, figsize=FIGSIZE, sharex=True)

t0 = pd.Timestamp("2020-01-10 00:00:00-0000")
t1 = pd.Timestamp("2020-01-15 00:00:00-0000")
t00 = t0 - pd.Timedelta(days=4)

# percolation_start_dt                           2019-04-24 16:00:00+00:00
# percolation_end_dt                             2019-08-05 08:00:00+00:00

# hourly rainfall
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1],
    color=LIGHT_BLUE,
    label="Rainfall", width=BWIDHT,
)

# moist 75
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l4_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l4_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    # linestyle="-.",
    linewidth=LINEWIDTH,
    label="75 mm",
)


ax[2].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l4_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l4_label, "Reading"].loc[t00:t1].mul(100),
    color=DARK_PURPLE, linestyle="-.",
)
# ax[2].plot(
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist280_l4_label, "Reading"].loc[t00:t1].index,
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist280_l4_label, "Reading"].loc[t00:t1].mul(100),
#     color='k', linestyle=":",
# )

ax[3].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1850_l4_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1850_l4_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    linestyle="-.",
    linewidth=LINEWIDTH,
    label="1850 mm",
)
ax[3].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1750_l4_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1750_l4_label, "Reading"].loc[t00:t1].mul(100),
    color=DARK_PURPLE,
    linestyle=":",
    linewidth=LINEWIDTH,
    label="1750 mm",
)


# # suction
# ax[1].plot(
#     data_soil_suction.loc[data_soil_suction["Unique_label_"] == suc1850_l4_label, "Reading"].loc[t00:t1].rolling(6).mean(),
#     color=GOLD_BROWN,
#     label="L4",
#     linewidth=LINEWIDTH,
# )



# ax[3].bar(
#     data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t0:t1].index,
#     data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, "Reading"].loc[t0:t1],
#     color=GOLD_BROWN, alpha=0.8,
#     label="L4", width=BWIDHT,
# )

# fonts and labels
ax[0].set_ylabel(r"Rainfall (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
ax[1].set_ylabel(r"Suction (kPa)", fontsize=FONTSIZE)
ax[2].set_ylabel(r"Percolation (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
ax[0].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82))
ax[1].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)
ax[2].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)

# increase lims
# ax[1].set_ylim(0, 5)

# # xticks only: 2020-10-01, 2020-10-15, 2020-11-01
# ax[0].set_xticks(
#     [
#         pd.Timestamp("2020-06-01 00:00:00-0000"),
#         pd.Timestamp("2020-07-01 00:00:00-0000"),
#         pd.Timestamp("2020-08-01 00:00:00-0000"),
#         pd.Timestamp("2020-09-01 00:00:00-0000"),
#     ]
# )

# # tick labels: year only for first tick, the rest: month-day
# ax[0].set_xticklabels(
#     [
#         "Jun 01 (2020)",
#         "Jul 01",
#         "Aug 01",
#         "Sep 01",
#     ]
# )

ax[1].yaxis.labelpad = 30
ax[2].yaxis.labelpad = 30

plt.tight_layout()
# plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=1000)
plt.show()

# Event L4 Fall

In [ ]:
SAVE_PATH = paths["fig_dir"] / f"2019L2Spring_Event19.pdf"
FIGSIZE = (19, 24)
FONTSIZE = 39
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 5
BWIDHT = 0.1

suc1850_l2_label = "L2_1850mm_int_Suction_kPa"
suc1750_l2_label = "L2__1750mm_int_Suction_kPa"
suc600_l2_label = "L2_600mm_int_Suction_kPa"
moist75_l2_label = "L2_75mm_ext_Moisture_percent"
moist225_l2_label = "L2_225mm_ext_Moisture_percent"
moist1850_l2_label = "L2_1850mm_int_Moisture_percent"
moist1750_l2_label = "L2_1750mm_int_Moisture_percent"
moist600_l2_label = "L2_600mm_int_Moisture_percent"

suc1850_l1_label = "L1_1850mm_int_Suction_kPa"
suc1750_l1_label = "L1_1750mm_int_Suction_kPa"
moist75_l1_label = "L1_75mm_ext_Moisture_percent"
moist225_l1_label = "L1_225mm_ext_Moisture_percent"

suc1850_l4_label = "L4_1850mm_int_Suction_kPa"
suc1750_l4_label = "L4_1750mm_int_Suction_kPa"
suc280_l4_label = "L4_280mm_int_Suction_kPa"
moist75_l4_label = "L4_75mm_ext_Moisture_percent"
moist225_l4_label = "L4_225mm_ext_Moisture_percent"
moist280_l4_label = "L4_280mm_int_Moisture_percent"
moist1850_l4_label = "L4_1850mm_int_Moisture_percent"
moist1750_l4_label = "L4_1750mm_int_Moisture_percent"

suc1850_l3_label = "L3_1850mm_int_Suction_kPa"
suc1750_l3_label = "L3_1750mm_int_Suction_kPa"
suc600_l3_label = "L3_600mm_int_Suction_kPa"
moist75_l3_label = "L3_75mm_ext_Moisture_percent"
moist225_l3_label = "L3_225mm_ext_Moisture_percent"
moist600_l3_label = "L3_600mm_int_Moisture_percent"
moist1850_l3_label = "L3_1850mm_int_Moisture_percent"
moist1750_l3_label = "L3_1750mm_int_Moisture_percent"

fig, ax = plt.subplots(4, 1, figsize=FIGSIZE, sharex=True)

t0 = pd.Timestamp("2020-09-30 20:00:00+00:00")
t1 = pd.Timestamp("2020-10-22 03:00:00+00:00")
t00 = t0 - pd.Timedelta(days=4)

# percolation_start_dt                           2019-05-06 21:00:00+00:00
# percolation_end_dt                             2019-05-15 03:00:00+00:00

# percolation_start_dt                           2019-04-24 15:00:00+00:00   L3
# percolation_end_dt                             2019-06-10 03:00:00+00:00

# hourly rainfall
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1],
    color=LIGHT_BLUE,
    label="Rainfall", width=BWIDHT,
)

# moist 75
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l2_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    # linestyle="-.",
    linewidth=LINEWIDTH,
    label="75 mm",
)


ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l2_label, "Reading"].loc[t00:t1].mul(100),
    color=GOLD_BROWN, linestyle="-.", linewidth=LINEWIDTH,
)
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist600_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist600_l2_label, "Reading"].loc[t00:t1].mul(100),
    color='k', linestyle=":",
)

# ax[2].plot(
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist280_l4_label, "Reading"].loc[t00:t1].index,
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist280_l4_label, "Reading"].loc[t00:t1].mul(100),
#     color='k', linestyle=":",
# )

ax[2].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1850_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1850_l2_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    linestyle="-.",
    linewidth=LINEWIDTH,
    label="1850 mm",
)
ax[2].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1750_l2_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1750_l2_label, "Reading"].loc[t00:t1].mul(100),
    color=DARK_PURPLE,
    linestyle=":",
    linewidth=LINEWIDTH,
    label="1750 mm",
)


# # suction
# ax[2].plot(
#     data_soil_suction.loc[data_soil_suction["Unique_label_"] == suc1850_l2_label, "Reading"].loc[t00:t1].rolling(6).mean(),
#     color=GOLD_BROWN,
#     label="L2",
#     linewidth=LINEWIDTH,
# )



ax[3].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, "Reading"].loc[t0:t1].index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, "Reading"].loc[t0:t1],
    color=LIGHT_BLUE,
    label="L2", width=BWIDHT,
)

# fonts and labels
ax[0].set_ylabel(r"Rainfall (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
ax[1].set_ylabel(r"Suction (kPa)", fontsize=FONTSIZE)
ax[2].set_ylabel(r"Percolation (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
ax[0].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82))
ax[1].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)
ax[2].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)

# increase lims
# ax[1].set_ylim(0, 5)

# # xticks only: 2019-05-02, 2019-05-06, 2019-05-15
# ax[0].set_xticks(
#     [
#         pd.Timestamp("2019-05-02 21:00:00-0000"),
#         pd.Timestamp("2019-05-06 21:00:00-0000"),
#         pd.Timestamp("2019-05-15 08:00:00-0000"),
#     ]
# )

# # tick labels: year only for first tick, the rest: month-day
# ax[0].set_xticklabels(
#     [
#         "2020-05-02",
#         "06", "15",
#     ]
# )

ax[1].yaxis.labelpad = 30
ax[0].yaxis.labelpad = 35
ax[2].tick_params(axis="x", which="major", pad=15)

# ax[2].set

plt.tight_layout()
# plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=1000)
plt.show()

In [ ]:
# SAVE_PATH = paths["fig_dir"] / f"2019L2Spring_Event19.pdf"
FIGSIZE = (19, 20)
FONTSIZE = 39
FONT_LEGEND = FONTSIZE - 6
LINEWIDTH = 5
BWIDHT = 0.1

suc1850_l2_label = "L2_1850mm_int_Suction_kPa"
suc1750_l2_label = "L2__1750mm_int_Suction_kPa"
suc600_l2_label = "L2_600mm_int_Suction_kPa"
moist75_l2_label = "L2_75mm_ext_Moisture_percent"
moist225_l2_label = "L2_225mm_ext_Moisture_percent"
moist1850_l2_label = "L2_1850mm_int_Moisture_percent"
moist1750_l2_label = "L2_1750mm_int_Moisture_percent"
moist600_l2_label = "L2_600mm_int_Moisture_percent"

suc1850_l1_label = "L1_1850mm_int_Suction_kPa"
suc1750_l1_label = "L1_1750mm_int_Suction_kPa"
moist75_l1_label = "L1_75mm_ext_Moisture_percent"
moist225_l1_label = "L1_225mm_ext_Moisture_percent"

suc1850_l4_label = "L4_1850mm_int_Suction_kPa"
suc1750_l4_label = "L4_1750mm_int_Suction_kPa"
suc280_l4_label = "L4_280mm_int_Suction_kPa"
moist75_l4_label = "L4_75mm_ext_Moisture_percent"
moist225_l4_label = "L4_225mm_ext_Moisture_percent"
moist280_l4_label = "L4_280mm_int_Moisture_percent"
moist1850_l4_label = "L4_1850mm_int_Moisture_percent"
moist1750_l4_label = "L4_1750mm_int_Moisture_percent"

suc1850_l3_label = "L3_1850mm_int_Suction_kPa"
suc1750_l3_label = "L3_1750mm_int_Suction_kPa"
suc600_l3_label = "L3_600mm_int_Suction_kPa"
moist75_l3_label = "L3_75mm_ext_Moisture_percent"
moist225_l3_label = "L3_225mm_ext_Moisture_percent"
moist600_l3_label = "L3_600mm_int_Moisture_percent"
moist1850_l3_label = "L3_1850mm_int_Moisture_percent"
moist1750_l3_label = "L3_1750mm_int_Moisture_percent"

fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

t0 = pd.Timestamp("2021-04-23 13:00:00+0000")
t1 = pd.Timestamp("2021-06-13 13:00:00+0000")
t00 = t0 - pd.Timedelta(days=4)

# percolation_start_dt                           2019-04-24 16:00:00+00:00    L4
# percolation_end_dt                             2019-08-05 08:00:00+00:00

# percolation_start_dt                           2021-04-23 13:00:00+00:00 L1
# percolation_end_dt                             2021-06-13 13:00:00+00:00


# hourly rainfall
ax[0].bar(
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1].index,
    data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Rainfall", "Reading"].loc[t00:t1],
    color=LIGHT_BLUE,
    label="Rainfall", width=BWIDHT,
)

# moist 75
ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l1_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist75_l1_label, "Reading"].loc[t00:t1].mul(100),
    color=LIGHT_BLUE,
    # linestyle="-.",
    linewidth=LINEWIDTH,
    label="75 mm",
)


ax[1].plot(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l1_label, "Reading"].loc[t00:t1].index,
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist225_l1_label, "Reading"].loc[t00:t1].mul(100),
    color=GOLD_BROWN, linestyle="-.", linewidth=LINEWIDTH,
)
# ax[1].plot(
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist600_l4_label, "Reading"].loc[t00:t1].index,
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist600_l4_label, "Reading"].loc[t00:t1].mul(100),
#     color='k', linestyle=":",
# )

# ax[2].plot(
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist280_l4_label, "Reading"].loc[t00:t1].index,
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist280_l4_label, "Reading"].loc[t00:t1].mul(100),
#     color='k', linestyle=":",
# )

# ax[1].plot(
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1850_l3_label, "Reading"].loc[t00:t1].index,
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1850_l3_label, "Reading"].loc[t00:t1].mul(100),
#     color=LIGHT_BLUE,
#     linestyle="-.",
#     linewidth=LINEWIDTH,
#     label="1850 mm",
# )
# ax[1].plot(
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1750_l2_label, "Reading"].loc[t00:t1].index,
#     data_soil_moist.loc[data_soil_moist["Unique_label_"] == moist1750_l2_label, "Reading"].loc[t00:t1].mul(100),
#     color=DARK_PURPLE,
#     linestyle=":",
#     linewidth=LINEWIDTH,
#     label="1750 mm",
# )


# # suction
# ax[1].plot(
#     data_soil_suction.loc[data_soil_suction["Unique_label_"] == suc1850_l4_label, "Reading"].loc[t00:t1].rolling(6).mean(),
#     color=GOLD_BROWN,
#     label="L4",
#     linewidth=LINEWIDTH,
# )



ax[2].bar(
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 1, "Reading"].loc[t0:t1].index,
    data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 1, "Reading"].loc[t0:t1],
    color=LIGHT_BLUE,
    label="L1", width=BWIDHT,
)

# fonts and labels
ax[0].set_ylabel(r"Rainfall (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)
ax[1].set_ylabel(r"Suction (kPa)", fontsize=FONTSIZE)
ax[2].set_ylabel(r"Percolation (mm $\mathregular{h^{-1}}$)", fontsize=FONTSIZE)

ax[0].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[1].tick_params(axis="both", which="major", labelsize=FONTSIZE)
ax[2].tick_params(axis="both", which="major", labelsize=FONTSIZE)

# legends
ax[0].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82))
ax[1].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)
ax[2].legend(fontsize=FONT_LEGEND, frameon=False, loc=(0.00, 0.82), ncol=2)

# put (a), ...
for ax_, letter in zip(ax, ["(a)", "(b)", "(c)"]):
    ax_.text(
        0.5,
        1.03,
        letter,
        transform=ax_.transAxes,
        size=FONTSIZE,
)

# increase lims
# ax[1].set_ylim(0, 5)

# # xticks only: 2019-05-02, 2019-05-06, 2019-05-15
# ax[0].set_xticks(
#     [
#         pd.Timestamp("2019-05-02 21:00:00-0000"),
#         pd.Timestamp("2019-05-06 21:00:00-0000"),
#         pd.Timestamp("2019-05-15 08:00:00-0000"),
#     ]
# )

# # tick labels: year only for first tick, the rest: month-day
# ax[0].set_xticklabels(
#     [
#         "2020-05-02",
#         "06", "15",
#     ]
# )

ax[1].yaxis.labelpad = 30
ax[0].yaxis.labelpad = 35
ax[2].tick_params(axis="x", which="major", pad=15)

# ax[2].set

plt.tight_layout()
# plt.savefig(SAVE_PATH, bbox_inches="tight", dpi=1000)
plt.show()

In [ ]:


# all of them
fig = px.line(
    data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L4_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean(),
    x=data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L4_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean().index,
    y="Reading",
    title="Suction",
)
fig.add_scatter(
    x=data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L2_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean().index,
    y=data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L2_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L2",
)
fig.add_scatter(
    x=data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L3_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean().index,
    y=data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L3_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L3",
)
fig.add_scatter(
    x=data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L1_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean().index,
    y=data_soil_suction.loc[data_soil_suction["Unique_label_"] == "L1_1850mm_int_Suction_kPa", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L1",
)

fig.show()


fig = px.line(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L4_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean(),
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L4_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean().index,
    y="Reading",
    title="Moisture",
)
fig.add_scatter(
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L2_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean().index,
    y=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L2_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L2",
)
fig.add_scatter(
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L3_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean().index,
    y=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L3_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L3",
)
fig.add_scatter(
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L1_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean().index,
    y=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L1_1850mm_int_Moisture_percent", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L1",
)
fig.show()


fig = px.line(
    data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L4_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean(),
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L4_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean().index,
    y="Reading",
    title="Moisture",
)
fig.add_scatter(
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L2_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean().index,
    y=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L2_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L2",
)
fig.add_scatter(
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L3_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean().index,
    y=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L3_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L3",
)
fig.add_scatter(
    x=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L1_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean().index,
    y=data_soil_moist.loc[data_soil_moist["Unique_label_"] == "L1_75mm_ext_Moisture_percent", "Reading"].resample("1d").mean(),
    mode="lines",
    name="L1",
)
fig.show()

In [ ]:
# data_events.loc["Event-19"]
# find events that start in spring
for event in data_events.index:
    if data_events.loc[event, "percolation_start_dt"].month in [3, 4, 5]:
        print(event)

In [ ]:
data_events.loc["Event-55"]

# Inspect individual events

## Helpers

In [ ]:
# Function to plot primary data (left y-axis)
def plot_primary(ax, data, label, color, width=None, linestyle='-'):

    global LINEWIDTH, FONTSIZE, FONT_LEGEND, SLATE_GREY, LIGHT_GREY

    if width:  # It's a bar plot
        ax.bar(data.index, data, color=color, width=width, label=label)
    else:  # It's a line plot
        ax.plot(data.index, data, color=color, linewidth=LINEWIDTH, label=label, linestyle=linestyle)
    ax.set_ylabel(label, fontsize=FONTSIZE)
    ax.tick_params(axis="both", which="major", labelsize=FONTSIZE)
    ax.legend(loc="upper left", fontsize=FONT_LEGEND)

# Function to plot secondary data (right y-axis)
def plot_secondary(ax, data, label, color, linestyle):

    global LINEWIDTH, FONTSIZE, FONT_LEGEND, SLATE_GREY, LIGHT_GREY

    ax2 = ax.twinx()
    ax2.plot(data.index, data, color=color, linewidth=LINEWIDTH, label=label, linestyle=linestyle)
    ax2.set_ylabel(label, fontsize=FONTSIZE)
    ax2.tick_params(axis="both", which="major", labelsize=FONTSIZE)
    ax2.legend(loc="upper right", fontsize=FONT_LEGEND)

    return ax2

def give_lys_labels(lysimeter_no: int ):

    surface_soil_moist_lab = None
    surface_soil_temp_lab = None
    lysimeter_lab = None
    lysimeter_nr = None

    match lysimeter_no:

        case "L1":
            surface_soil_moist_lab = "L1_75mm_ext_Moisture_percent"
            surface_soil_temp_lab = "L1_75mm_ext_Temperature_degc"
            lysimeter_lab = "Enclosure1_Lysimeter1_percolation_mmh"
            lysimeter_nr = 1

        case "L2":
            surface_soil_moist_lab = "L2_75mm_ext_Moisture_percent"
            surface_soil_temp_lab = "L2_75mm_ext_Temperature_degc"
            lysimeter_lab = "Enclosure1_Lysimeter2_percolation_mmh"
            lysimeter_nr = 2

        case "L3":
            surface_soil_moist_lab = "L3_75mm_ext_Moisture_percent"
            surface_soil_temp_lab = "L3_75mm_ext_Temperature_degc"
            lysimeter_lab = "Enclosure2_Lysimeter3_percolation_mmh"
            lysimeter_nr = 3

        case "L4":
            surface_soil_moist_lab = "L4_75mm_ext_Moisture_percent"
            surface_soil_temp_lab = "L4_75mm_ext_Temperature_degc"
            lysimeter_lab = "Enclosure3_Lysimeter4_percolation_mmh"
            lysimeter_nr = 4

        case _:
            raise ValueError(
                "Invalid lysimeter name. Must be one of `L1`, `L2`, `L3`, `L4`"
            )

    return surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab, lysimeter_nr

def get_time_vars(event_id: str):

    global percolation_events, pre_event_window_hours, LOCAL_TZ

    t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
    t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
    t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
    t1 = t1 + pd.Timedelta(hours=48)

    return t0, t00, t1

## Snow-affected events

### tabular report

In [ ]:
# Events characteristics
percolation_due_to_rain_on_snow = ["Event-4", "Event-5", "Event-6", "Event-7", "Event-9", "Event-10", "Event-11", "Event-12",
                                   "Event-13", "Event-14", "Event-26", "Event-29", 'Event-31', "Event-35", "Event-36", "Event-37",
                                   "Event-59", "Event-60",
                                   'Event-32', 'Event-39', 'Event-40', 'Event-41',
                                   "Event-54", "Event-63",
                                ]
percolation_before_soil_reaction = ["Event-7", "Event-9", "Event-10", "Event-13", "Event-14", "Event-8", #"Event-15",
                                    "Event-27", "Event-28", "Event-30", 'Event-31', "Event-33", "Event-34",
                                    'Event-32',
                                ]

percolation_snowmelt_only = ["Event-8", 'Event-16', "Event-27", "Event-28", "Event-33", "Event-34", "Event-38", "Event-61",
                                   "Event-62"]
percolation_with_frozen_soil = ["Event-9", "Event-13", "Event-14", "Event-27", "Event-28", "Event-29", "Event-30",
                                "Event-33", "Event-34", "Event-35", "Event-36", "Event-37", "Event-61", "Event-62",
                                'Event-31', 'Event-32',
                            ]

# Event IDs
event_ids = sorted(set(percolation_due_to_rain_on_snow + percolation_before_soil_reaction +
                       percolation_snowmelt_only + percolation_with_frozen_soil), key=lambda x: int(x.split('-')[1]))

# Compile characteristics into a dictionary
event_characteristics = {
    "Event ID": event_ids,
    "Percolation due to rain-on-snow": ["Yes" if event in percolation_due_to_rain_on_snow else "" for event in event_ids],
    "Percolation snowmelt only": ["Yes" if event in percolation_snowmelt_only else "" for event in event_ids],
    "Percolation before soil reaction": ["Yes" if event in percolation_before_soil_reaction else ("Less Likely" if event == "Event-12" else "") for event in event_ids],
    "Percolation with frozen soil": ["Yes" if event in percolation_with_frozen_soil else ("Maybe" if event == "Event-61" else ("Based on L4" if event in ["Event-62", "Event-35", "Event-37"] else "")) for event in event_ids]
}

# Create DataFrame
df = pd.DataFrame(event_characteristics)

# Set Event ID as index
df.set_index("Event ID", inplace=True)

df


In [ ]:
# there are 24 events where we have rain-on-snow before percolation start
# 9 events where snowmelt is the only source of water before the percolation start

### Event-4

In [ ]:
event_id = "Event-4"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event-5

In [ ]:
event_id = "Event-5"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event-6

In [ ]:
event_id = "Event-6"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 7

In [ ]:
event_id = "Event-7"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
plot_primary(ax[0], svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"], "Water input", LIGHT_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

In [ ]:
data_hhourly_soil.loc[t00:t1, "L1_225mm_ext_Moisture_percent"].plot(figsize=(15, 9))
data_hhourly_soil.loc[t00:t1, "L1_1750mm_int_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, "L1_1750mm_ext_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, "L1_1850mm_int_Moisture_percent"].plot()

# vertical at t0
plt.axvline(x=t0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.legend()

### Event-8

In [ ]:
uf.print_event_info(percolation_events.loc["Event-8"])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4

event_id = "Event-8"
surface_soil_moist_lab = "L1_75mm_ext_Moisture_percent"
surface_soil_temp_lab = "L1_75mm_ext_Temperature_degc"
lysimeter_lab = "Enclosure1_Lysimeter1_percolation_mmh"

# Start: 2018-11-26 10:00:00-05:00
# End: 2018-11-27 05:00:00-05:00
t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
t1 = t1 + pd.Timedelta(hours=48)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

In [ ]:
percolation_events.loc["Event-8"]

### Event 9

In [ ]:
uf.print_event_info(percolation_events.loc["Event-9"])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4

event_id = "Event-9"
surface_soil_moist_lab = "L4_75mm_ext_Moisture_percent"
surface_soil_temp_lab = "L4_75mm_ext_Temperature_degc"
lysimeter_lab = "Enclosure3_Lysimeter4_percolation_mmh"

# Start: 2018-11-26 10:00:00-05:00
# End: 2018-11-27 05:00:00-05:00
t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
t1 = t1 + pd.Timedelta(hours=48)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
plot_primary(ax[0], svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"], "Water input", LIGHT_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 10

In [ ]:
event_id = "Event-10"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab = "L2_75mm_ext_Moisture_percent"
surface_soil_temp_lab = "L2_75mm_ext_Temperature_degc"
lysimeter_lab = "Enclosure1_Lysimeter2_percolation_mmh"

# Start: 2018-11-26 10:00:00-05:00
# End: 2018-11-27 05:00:00-05:00
t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
t1 = t1 + pd.Timedelta(hours=48)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 11

In [ ]:
event_id = "Event-11"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab = "L3_75mm_ext_Moisture_percent"
surface_soil_temp_lab = "L3_75mm_ext_Temperature_degc"
lysimeter_lab = "Enclosure2_Lysimeter3_percolation_mmh"

# Start: 2018-11-26 10:00:00-05:00
# End: 2018-11-27 05:00:00-05:00
t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
t1 = t1 + pd.Timedelta(hours=48)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 12

In [ ]:
event_id = "Event-12"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab = "L3_75mm_ext_Moisture_percent"
surface_soil_temp_lab = "L3_75mm_ext_Temperature_degc"
lysimeter_lab = "Enclosure2_Lysimeter3_percolation_mmh"

# Start: 2018-11-26 10:00:00-05:00
# End: 2018-11-27 05:00:00-05:00
t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
t1 = t1 + pd.Timedelta(hours=48)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

In [ ]:
data_hhourly_soil.loc[t00:t1, ]["L3_225mm_ext_Moisture_percent"].plot(figsize=(15, 9))
data_hhourly_soil.loc[t00:t1, ]["L3_600mm_int_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, ]["L3_600mm_ext_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, ]["L3_1750mm_int_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, ]["L3_1850mm_int_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, ]["L3_1750mm_ext_Moisture_percent"].plot()




plt.legend()

### Event 13

In [ ]:
event_id = "Event-13"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab = "L4_75mm_ext_Moisture_percent"
surface_soil_temp_lab = "L4_75mm_ext_Temperature_degc"
lysimeter_lab = "Enclosure3_Lysimeter4_percolation_mmh"

# Start: 2018-11-26 10:00:00-05:00
# End: 2018-11-27 05:00:00-05:00
t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
t1 = t1 + pd.Timedelta(hours=48)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 14

In [ ]:
event_id = "Event-14"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab = "L3_75mm_ext_Moisture_percent"
surface_soil_temp_lab = "L3_75mm_ext_Temperature_degc"
lysimeter_lab = "Enclosure2_Lysimeter3_percolation_mmh"

# Start: 2018-11-26 10:00:00-05:00
# End: 2018-11-27 05:00:00-05:00
t0 = percolation_events.loc[event_id]["percolation_start_dt"].tz_convert(LOCAL_TZ)
t00 = t0 - pd.Timedelta(hours=pre_event_window_hours)
t1 = percolation_events.loc[event_id]["percolation_end_dt"].tz_convert(LOCAL_TZ)
t1 = t1 + pd.Timedelta(hours=48)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

In [ ]:
data_hhourly_soil.loc[t00:t1, :]["L3_225mm_ext_Moisture_percent"].plot(figsize=(15, 9))
data_hhourly_soil.loc[t00:t1, :]["L3_1750mm_int_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, :]["L3_1750mm_int_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, :]["L3_1750mm_ext_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1, :]["L3_1850mm_int_Moisture_percent"].plot()
plt.legend()

### Event 15

In [ ]:
event_id = "Event-15"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 16

In [ ]:
event_id = "Event-16"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 22

In [ ]:
event_id = "Event-22"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.05)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.075)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 24

In [ ]:
event_id = "Event-24"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 35
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.05)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.075)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 26

In [ ]:
event_id = "Event-26"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 27

In [ ]:
event_id = "Event-27"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
plot_primary(ax[0], svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"], "Water input", LIGHT_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

In [ ]:
svs_ens_ave.loc[t00:t0, "Water input at surface (mm/h)"].sum()

### Event 28

In [ ]:
event_id = "Event-28"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 29

In [ ]:
event_id = "Event-29"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 30

In [ ]:
event_id = "Event-30"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 31

In [ ]:
event_id = "Event-31"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 32

In [ ]:
event_id = "Event-32"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 33

In [ ]:
event_id = "Event-33"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.04)
plot_primary(ax[0], svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"], "Water input", LIGHT_GREY, width=0.03)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

In [ ]:
# svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"].sum()
# data_hourly_meteo.loc[t00:t1, ].loc[data_hourly_meteo["Air temperature (degC)"] > 0]["Precipitation (mm)"].sum()
data_hhourly_soil.loc[t00:t1]["L1_1850mm_int_Moisture_percent"].plot()


### Event 34

In [ ]:
event_id = "Event-34"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
plot_primary(ax[0], svs_ens_ave.loc[t00:t1, "Water input at surface (mm/h)"], "Water input", LIGHT_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

In [ ]:
# data_hhourly_soil.loc[t00:t1]["L4_1850mm_int_Moisture_percent"].plot()
# data_hhourly_soil.loc[t00:t1]["L4_1750mm_int_Moisture_percent"].plot()
# data_hhourly_soil.loc[t00:t1]["L4_225mm_ext_Moisture_percent"].plot()
data_hhourly_soil.loc[t00:t1]["L4_280mm_int_Moisture_percent"].plot()


In [ ]:
data_suction = pd.read_csv(
    "/Users/alireza_amani/Desktop/PhD_files/Efforts/DeepPercolationManuscript/Data/suction_all_sensors_hhly.csv",
    index_col=0, parse_dates=True
)

In [ ]:
data_suction.loc[t00:t1].isna().sum()
# data_suction.loc[t00:t1].shape


### Event35

In [ ]:
event_id = "Event-35"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 36

In [ ]:
event_id = "Event-36"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 37

In [ ]:
event_id = "Event-37"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event38

In [ ]:
event_id = "Event-38"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 39

In [ ]:
event_id = "Event-39"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.055)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.06)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 40

In [ ]:
event_id = "Event-40"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.055)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.06)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 41

In [ ]:
event_id = "Event-41"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.055)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.06)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 51

In [ ]:
event_id = "Event-51"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.055)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.06)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 54

In [ ]:
event_id = "Event-54"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.055)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.06)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### event 59

In [ ]:
event_id = "Event-59"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()

### Event 60

In [ ]:
event_id = "Event-60"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()


### Event 61 

In [ ]:
event_id = "Event-61"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()


### Event62

In [ ]:
event_id = "Event-62"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.025)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.05)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()


### Event 63

In [ ]:
event_id = "Event-63"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0], data_hourly_meteo.loc[t00:t1, "Rainfall (mm)"], "Rainfall", SLATE_GREY, width=0.05)
tax0 = plot_secondary(ax[0], data_daily_snowdepth.loc[t00:t1, "Snow on Grnd (cm)"], "Snow depth", LIGHT_GREY, '-')

plot_primary(ax[1], data_hourly_percolation.loc[t00:t1, lysimeter_lab], "Percolation", SLATE_GREY, width=0.06)
tax1 = plot_secondary(ax[1], data_hourly_meteo.loc[t00:t1, "Air temperature (degC)"], "Air temperature", LIGHT_GREY, 'dashdot')

plot_primary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_moist_lab], "Soil moisture", SLATE_GREY)
tax2 = plot_secondary(ax[2], data_hhourly_soil.loc[t00:t1, surface_soil_temp_lab], "Soil temperature", LIGHT_GREY, 'dashdot')

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()


## ..

In [ ]:
event_id = "Event-43"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab, lysimeter_nr = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0],
             data_meteo_hourly.loc[data_meteo_hourly.Variable == 'Rainfall'].loc[t00:t1, "Reading"],
            "Rainfall", SLATE_GREY, width=0.25
)
tax0 = plot_secondary(ax[0],
                      data_meteo_daily.loc[data_meteo_daily.Variable == 'Snow depth'].loc[t00.date():t1.date(), "Reading"],
                    "Snow depth", LIGHT_GREY, '-'
)

plot_primary(ax[1],
             data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == lysimeter_nr].loc[t00:t1, 'Reading'],
            "Percolation", SLATE_GREY, width=0.05
)
tax1 = plot_secondary(ax[1],
                      data_meteo_hourly.loc[data_meteo_hourly.Variable == 'Air temperature'].loc[t00:t1, "Reading"],
                      "Air temperature", LIGHT_GREY, 'dashdot'
)

plot_primary(ax[2],
            data_soil_moist.loc[data_soil_moist["Unique_label_"] == surface_soil_moist_lab].loc[t00:t1, "Reading"],
            "Soil moisture", SLATE_GREY
)
tax2 = plot_secondary(ax[2],
                      data_soil_temp.loc[data_soil_temp["Unique_label_"] == surface_soil_temp_lab].loc[t00:t1, "Reading"],
                      "Soil temperature", LIGHT_GREY, 'dashdot'
)

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()


In [ ]:
event_id = "Event-43"
uf.print_event_info(percolation_events.loc[event_id])

# constants
FIGSIZE = (24, 19)
FONTSIZE = 30
FONT_LEGEND = FONTSIZE - 12
LINEWIDTH = 4


surface_soil_moist_lab, surface_soil_temp_lab, lysimeter_lab, lysimeter_nr = give_lys_labels(
    percolation_events.loc[event_id]["event_label"][:2]
)

t0, t00, t1 = get_time_vars(event_id)

# Create figure and axes
fig, ax = plt.subplots(3, 1, figsize=FIGSIZE, sharex=True)

# Plot data
plot_primary(ax[0],
             data_meteo_hourly.loc[data_meteo_hourly.Variable == 'Rainfall'].loc[t00:t1, "Reading"],
            "Rainfall", SLATE_GREY, width=0.25
)
tax0 = plot_secondary(ax[0],
                      data_meteo_daily.loc[data_meteo_daily.Variable == 'Snow depth'].loc[t00.date():t1.date(), "Reading"],
                    "Snow depth", LIGHT_GREY, '-'
)

plot_primary(ax[1],
             data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == lysimeter_nr].loc[t00:t1, 'Reading'],
            "Percolation", SLATE_GREY, width=0.05
)
tax1 = plot_secondary(ax[1],
                      data_meteo_hourly.loc[data_meteo_hourly.Variable == 'Air temperature'].loc[t00:t1, "Reading"],
                      "Air temperature", LIGHT_GREY, 'dashdot'
)

plot_primary(ax[2],
            data_soil_moist.loc[data_soil_moist["Unique_label_"] == surface_soil_moist_lab].loc[t00:t1, "Reading"],
            "Soil moisture", SLATE_GREY
)
tax2 = plot_secondary(ax[2],
                      data_soil_temp.loc[data_soil_temp["Unique_label_"] == surface_soil_temp_lab].loc[t00:t1, "Reading"],
                      "Soil temperature", LIGHT_GREY, 'dashdot'
)

#Set xlabel for the bottom plot
ax[2].set_xlabel("")
ax[0].set_xticks(ax[0].get_xticks()[::2])

#For soil temp and air temp, plot horizontal line at 0
for tax in [tax1, tax2]:
    tax.axhline(y=0, color="black", linestyle="--", linewidth=LINEWIDTH/4)

plt.tight_layout()
plt.show()


In [ ]:
t0 = pd.Timestamp("2020-06-01 00:00:00", tz="America/Montreal")
t1 = pd.Timestamp("2020-09-30 23:59:59", tz="America/Montreal")

data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 1, 'Reading'
                            ].loc[t0:t1].plot(figsize=(24, 10))

data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, 'Reading'
                            ].loc[t0:t1].plot()

In [ ]:
t0, t00

# Plot individual events

In [ ]:
# plotly percolation from all lysimeters

for lysimeter_nr in data_hourly_percolation["Lysimeter Number"].unique():

    fig = go.Figure(layout=go.Layout(title=f"Lysimeter {lysimeter_nr} percolation",
                                     xaxis=dict(title=""),
                                     yaxis=dict(title="Percolation (mm)")))

    daily_ts = data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == lysimeter_nr]
    daily_ts = daily_ts.resample("1D").agg(
        # only if non-NaN values are present
        Reading=("Reading", lambda x: x.sum() if not x.isna().all() else np.nan)
    )
    fig.add_trace(go.Bar(
        x=daily_ts.index,
        y=daily_ts['Reading'],
        name=f'Lysimeter {lysimeter_nr}'
        ))

    # line
    fig.add_trace(go.Scatter(
        x=daily_ts.index,
        y=daily_ts['Reading'],
        mode='lines',
        name=f'Lysimeter {lysimeter_nr}'
    ))

    # increase width to make bars touch
    fig.update_traces(marker_line_width=0)
    fig.show()

In [ ]:
# total percolation from L3 during August 2020
data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 3, 'Reading'
                            ].loc["2020-08-01":"2020-08-31"].sum(), data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, 'Reading'
                            ].loc["2020-08-01":"2020-08-31"].sum(),

In [ ]:
# print Jul 2018 to Jan 2020, total percolation for L2, L3, L4
t0 = pd.Timestamp("2018-07-01 00:00:00", tz="America/Montreal")
t1 = pd.Timestamp("2020-01-31 23:59:59", tz="America/Montreal")

for lysimeter_nr in [2, 3, 4]:
    pts = data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == lysimeter_nr, 'Reading'
                                        ].loc[t0:t1].sum()
    print(f"Lysimeter {lysimeter_nr} total percolation from {t0} to {t1}: {pts:.2f} mm")
print("\n")

# Jan 2020 to Nov 2020, total percolation for L1, L2, L4
t0 = pd.Timestamp("2020-01-01 00:00:00", tz="America/Montreal")
t1 = pd.Timestamp("2020-11-30 23:59:59", tz="America/Montreal")

for lysimeter_nr in [1, 2, 4]:
    pts = data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == lysimeter_nr, 'Reading'
                                        ].loc[t0:t1].sum()
    print(f"Lysimeter {lysimeter_nr} total percolation from {t0} to {t1}: {pts:.2f} mm")


In [ ]:
# lets plot seasonal volume of percolation for L2 and precipitation
t0 = pd.Timestamp("2018-06-01 00:00:00", tz="America/Montreal")
t1 = pd.Timestamp("2020-08-31 23:59:59", tz="America/Montreal")

precipitation_ts = data_meteo_hourly.loc[data_meteo_hourly["Variable"] == "Precipitation", 'Reading'
                                            ].loc[t0:t1]

precipitation_ts = pd.DataFrame({
    "datetime": precipitation_ts.index,
    "precipitation": precipitation_ts.values
})

precipitation_ts['month'] = precipitation_ts['datetime'].dt.month
precipitation_ts['season'] = precipitation_ts['month'].apply(lambda x: "Winter" if x in [12, 1, 2] else
                                                            "Spring" if x in [3, 4, 5] else
                                                            "Summer" if x in [6, 7, 8] else
                                                            "Fall")
precipitation_ts['year'] = precipitation_ts['datetime'].dt.year

# L2
percolation_ts = data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 2, 'Reading'
                                                ].loc[t0:t1]
percolation_ts = pd.DataFrame({
    "datetime": percolation_ts.index,
    "percolation": percolation_ts.values
})

percolation_ts['month'] = percolation_ts['datetime'].dt.month
percolation_ts['season'] = percolation_ts['month'].apply(lambda x: "Winter" if x in [12, 1, 2] else
                                                        "Spring" if x in [3, 4, 5] else
                                                        "Summer" if x in [6, 7, 8] else
                                                        "Fall")
percolation_ts['year'] = percolation_ts['datetime'].dt.year

# percolation_ts.groupby(['year', 'season'])['percolation'].sum()

# L4
percolation_l4_ts = data_hourly_percolation.loc[data_hourly_percolation["Lysimeter Number"] == 4, 'Reading'
                                                ].loc[t0:t1]
percolation_l4_ts = pd.DataFrame({
    "datetime": percolation_l4_ts.index,
    "percolation": percolation_l4_ts.values
})

percolation_l4_ts['month'] = percolation_l4_ts['datetime'].dt.month
percolation_l4_ts['season'] = percolation_l4_ts['month'].apply(lambda x: "Winter" if x in [12, 1, 2] else
                                                        "Spring" if x in [3, 4, 5] else
                                                        "Summer" if x in [6, 7, 8] else
                                                        "Fall")
percolation_l4_ts['year'] = percolation_l4_ts['datetime'].dt.year

percolation_l4_ts.groupby(['year', 'season'])['percolation'].sum()
percolation_ts.groupby(['year', 'season'])['percolation'].sum()


In [ ]:
precipitation_ts.groupby(['year', 'season']).agg(
    precipitation=("precipitation", "sum")
)